In [34]:
import akshare as ak

# df = ak.tool_trade_date_hist_sina()

df = ak.fund_new_found_em()


In [35]:
from pretty_html_table import build_table

html_table_blue_light = build_table(df, 'grey_light')

# Save to html file
with open('pretty_table.html', 'w') as f:
    f.write(html_table_blue_light)



In [40]:
dct = df.head().to_dict()
import pandas as pd
# pd.DataFrame(dct)
dct

{'基金代码': {0: '018216', 1: '018201', 2: '018200', 3: '018182', 4: '018183'},
 '基金简称': {0: '长城久恒灵活配置混合C',
  1: '嘉实恒生消费指数发起(QDII)C',
  2: '嘉实恒生消费指数发起(QDII)A',
  3: '万家中证软件服务指数发起式A',
  4: '万家中证软件服务指数发起式C'},
 '发行公司': {0: '长城基金', 1: '嘉实基金', 2: '嘉实基金', 3: '万家基金', 4: '万家基金'},
 '基金类型': {0: '混合型-灵活', 1: 'QDII', 2: 'QDII', 3: '指数型-股票', 4: '指数型-股票'},
 '集中认购期': {0: '23/04/03～23/04/03',
  1: '23/03/30～23/04/06',
  2: '23/03/30～23/04/06',
  3: '23/03/27～23/03/27',
  4: '23/03/27～23/03/27'},
 '募集份额': {0: nan, 1: 0.13, 2: 0.13, 3: 0.1, 4: 0.1},
 '成立日期': {0: datetime.date(2023, 4, 3),
  1: datetime.date(2023, 4, 11),
  2: datetime.date(2023, 4, 11),
  3: datetime.date(2023, 3, 29),
  4: datetime.date(2023, 3, 29)},
 '成立来涨幅': {0: 7.79, 1: 0.07, 2: 0.07, 3: -7.26, 4: -7.27},
 '基金经理': {0: '储雯玉', 1: '刘珈吟', 2: '刘珈吟', 3: '杨坤', 4: '杨坤'},
 '申购状态': {0: '开放申购', 1: '开放申购', 2: '开放申购', 3: '开放申购', 4: '开放申购'},
 '优惠费率': {0: 0.0, 1: 0.0, 2: 0.1, 3: 0.1, 4: 0.0}}

In [5]:
# 判断是否需要更新数据
from datetime import datetime
now_str = datetime.now().strftime('%Y-%m-%d')

tool_trade_date_hist_sina_df

,trade_date
0,1990-12-19
1,1990-12-20
2,1990-12-21
3,1990-12-24
4,1990-12-25
...,...
8065,2023-12-25
8066,2023-12-26
8067,2023-12-27
8068,2023-12-28


In [28]:
from bisect import bisect_right
tool_trade_date_hist_sina_df['trade_date'] = tool_trade_date_hist_sina_df['trade_date'].astype(str)
idx = bisect_right(tool_trade_date_hist_sina_df['trade_date'].tolist(), now_str)
val = tool_trade_date_hist_sina_df.iloc[idx - 1].to_dict().get('trade_date')

lastest_date_str = datetime.now().strftime('%Y-%m-%d-%H')
last_update = val + '-15'
first_update = val + '-10'

flag = lastest_date_str >= last_update or lastest_date_str <= first_update

flag

True

In [ ]:
import json
import smtplib
from email.mime.text import MIMEText
from email.header import Header
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart


html_msg = pd.DataFrame(dct).to_html()
html_msg= "<html>" + html_msg + "</html>"


key = 'letadehhnxtpbdca'
EMAIL_ADDRESS = 'wikizeros@qq.com'
EMAIL_PASSWORD = key
context = ssl.create_default_context()
sender = EMAIL_ADDRESS
receiver = ['vance.coder@139.com']

subject = "subject here"
body = 'test html table'
msg = MIMEText()
msg['subject'] = subject
msg['From'] = sender
msg['To'] = receiver
msg.set_content(body)

# msg.attach(MIMEText(html_msg, "html", "utf-8"))

with smtplib.SMTP_SSL("smtp.qq.com", 465, context=context) as smtp:
    # smtp.set_debuglevel(1)
    smtp.ehlo('smtp.qq.com')
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    smtp.send_message(msg)
    


In [119]:
import pymongo
from datetime import datetime

# docker pull mongo:latest
# docker run -itd --name mongo -p 27017:27017 mongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["dbs"]
quo_col = mydb.quotation
concept = mydb.concept

In [102]:
import akshare as ak
symbol='600418'

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date="20220810", end_date="20230512",
                                            adjust="qfq")

dct = stock_zh_a_hist_df.to_dict(orient='records')
now_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
data = {
    '_id':symbol,
    'time': now_datetime,
    'data': dct
}


In [103]:
quo_col.update_one({'_id': symbol}, {'$set': data}, upsert=True)

In [118]:
quo_col.find_one({'_id': symbol})['time']

'2023-05-13 00:51:21'

In [ ]:
concept.find_one()